In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests as rq
import re

# bring subject's code

In [2]:
req = rq.get("https://cdcs.ur.rochester.edu/")
html = req.text
soup = BeautifulSoup(html, "lxml")

In [3]:
tables=soup.find("select", tabindex= "3")


In [4]:
sbjs=tables.find_all("option")

In [5]:
subjects = []
for sbj in sbjs:
    subjects.append(sbj.text)

In [6]:
subjects.pop(0)

''

In [7]:
pattern = "[A-Z]{2,3}"

In [8]:
clean_sub = []
for i in range(0, len(subjects)):
    match = re.findall(pattern, subjects[i])
    clean_sub.append(match[0])

# make Crawling CDCS fuction

In [9]:
def Crawling_cdcs(sub):
    # bring data from CDCS
    url = "https://cdcs.ur.rochester.edu/Query.aspx?id=DARS&subj="+sub
    req = rq.get(url)
    html = req.text
    soup = BeautifulSoup(html, "lxml")
    tables=soup.find_all("table", width = "100%")
    
    # 
    num = 1
    sub =pd.DataFrame(columns = ["Course", "Term","Day", "Begin", "End", "Building", "Room"])

    for a in tables:


        refresh = [[]]
        refresh2 = [[]]
        string = str(num)
        if num < 10:
            string = "0"+string
        # Course 
        Course=("rpResults_ctl"+string+"_lblCNum")
        Term = ("rpResults_ctl"+string+"_lblTerm")
        status = ("rpResults_ctl"+string+"_lblStatus")
        # 스케줄 
        
        # 수업 데이터 없는거 스킵
        nineteen = a.find("span", id = Term).text
        if a.find("td", width = "20%") == None:
            num += 2
            continue
        # 2020 만 가져오기 
        if "2020" not in nineteen:
            num += 2
            continue
        Status = a.find("span", id = status)
        # cancelled or closed skip
        if "Cancelled" in Status or "Closed" in Status:
            num +=2 
            continue
        # 데이터 정리 
        refresh[0].append(a.find("span", id = Course).text)
        refresh[0].append(a.find("span", id = Term).text)
        schedule =a.find_all("td", width = "20%")
        if len(schedule) <= 5:
            for s in schedule:
                refresh[0].append(s.text.strip())
            result0 = pd.DataFrame(data = refresh,columns = ["Course", "Term","Day", "Begin", "End", "Building", "Room"])
            sub = sub.append(result0, ignore_index= True)
        else:
            refresh2[0].append(a.find("span", id = Course).text)
            refresh2[0].append(a.find("span", id = Term).text)
            d = 0
            while d < 5:
                s = schedule[d]
                refresh[0].append(s.text.strip())
                d += 1
            while d < 10 and d>=5:
                s = schedule[d]
                refresh2[0].append(s.text.strip())
                d+=1
            result0 = pd.DataFrame(data = refresh,columns = ["Course", "Term","Day", "Begin", "End", "Building", "Room"])
            result1 = pd.DataFrame(data = refresh2,columns = ["Course", "Term","Day", "Begin", "End", "Building", "Room"])
            sub = sub.append(result0, ignore_index = True)
            sub = sub.append(result1, ignore_index = True)
        
        num += 2 
            
        
    return sub

In [10]:
Ult_subject = {}
for sub in clean_sub:
    Ult_subject[sub] = Crawling_cdcs(sub)

SSLError: ("read error: Error([('SSL routines', 'ssl3_get_record', 'decryption failed or bad record mac')])",)

In [12]:
Ult_subject["CSC"]

,Course,Term,Day,Begin,End,Building,Room
0,CSC 161,Spring 2020,TR,1650,1805,HOYT,AUD
1,CSC 161,Spring 2020,MW,1815,1930,HARK,114
2,CSC 161,Spring 2020,TR,1400,1515,GAVET,208
3,CSC 161,Spring 2020,MW,1525,1640,GAVET,244
4,CSC 161,Spring 2020,TR,1230,1345,GAVET,208
5,CSC 161,Spring 2020,TR,1815,1930,GAVET,244
6,CSC 161,Spring 2020,W,1815,1930,LATT,431
7,CSC 161,Spring 2020,T,1940,2055,WEGMN,1009
8,CSC 161,Spring 2020,M,1650,1805,MEL,206
9,CSC 161,Spring 2020,W,1940,2055,MEL,209


# prerequisite 추가 버전

In [11]:
def Crawling_cdcs(sub):
    # bring data from CDCS
    url = "https://cdcs.ur.rochester.edu/Query.aspx?id=DARS&subj="+sub
    req = rq.get(url)
    html = req.text
    soup = BeautifulSoup(html, "lxml")
    tables=soup.find_all("table", width = "100%")
    
    # 
    num = 1
    sub =pd.DataFrame(columns = ["Course", "Term","Day", "Begin", "End", "Building", "Room","Prerequisites"])

    for a in tables:


        refresh = [[]]
        refresh2 = [[]]
        string = str(num)
        if num < 10:
            string = "0"+string
        # address
        Course=("rpResults_ctl"+string+"_lblCNum")
        Term = ("rpResults_ctl"+string+"_lblTerm")
        status = ("rpResults_ctl"+string+"_lblStatus")
        pre = ("rpResults_ctl"+string+"_lblPrerequisites")
        # 스케줄 
        
        # 수업 데이터 없는거 스킵
        nineteen = a.find("span", id = Term).text
        if a.find("td", width = "20%") == None:
            num += 2
            continue
        # 2019 만 가져오기 
        if "2019" not in nineteen:
            num += 2
            continue
        Status = a.find("span", id = status)
        # cancelled or closed skip
        if "Cancelled" in Status or "Closed" in Status:
            num +=2 
            continue
        # 데이터 정리 
        # Course and Term
        refresh[0].append(a.find("span", id = Course).text)
        refresh[0].append(a.find("span", id = Term).text)
        schedule =a.find_all("td", width = "20%")
        
        
        # pre를 위한 정규표현식
        pattern2 = re.compile("[A-Z]{3}\s?\d{3}[A-Z]?")
        
        #  1 줄일때
        if len(schedule) <= 5:
            for s in schedule:
                # 시간표 & 장소
                refresh[0].append(s.text.strip())
            # Prerequisites
            if a.find("span", id = pre) == None:
                refresh[0].append("None")
            else:
                prere=a.find("span", id = pre).text
                refresh[0].append(pattern2.findall(prere))
            result0 = pd.DataFrame(data = refresh,columns = ["Course", "Term","Day", "Begin", "End", "Building", "Room","Prerequisites"])
            sub = sub.append(result0, ignore_index= True)
            
            
        #  2 줄일때  
        else:
            # Course and Term
            refresh2[0].append(a.find("span", id = Course).text)
            refresh2[0].append(a.find("span", id = Term).text)
            d = 0
            #  시간표 & 장소
            while d < 5:
                s = schedule[d]
                refresh[0].append(s.text.strip())
                d += 1
            while d < 10 and d>=5:
                s = schedule[d]
                refresh2[0].append(s.text.strip())
                d+=1
            # Prerequisites 1 줄 
            if a.find("span", id = pre) == None:
                refresh[0].append("None")
            else:
                prere=a.find("span", id = pre).text
                refresh[0].append(pattern2.findall(prere))
               
            # for  pre 2번째 줄 
            if a.find("span", id = pre) == None:
                refresh2[0].append("None")
            else:
                prere=a.find("span", id = pre).text
                refresh2[0].append(pattern2.findall(prere))
                
            
            result0 = pd.DataFrame(data = refresh,columns = ["Course", "Term","Day", "Begin", "End", "Building", "Room","Prerequisites"])
            result1 = pd.DataFrame(data = refresh2,columns = ["Course", "Term","Day", "Begin", "End", "Building", "Room","Prerequisites"])
            sub = sub.append(result0, ignore_index = True)
            sub = sub.append(result1, ignore_index = True)
        
        num += 2 
            
        
    return sub

# Cross list 추가버전 

In [10]:
def Crawling_cdcs(sub):
    # bring data from CDCS
    url = "https://cdcs.ur.rochester.edu/Query.aspx?id=DARS&subj="+sub
    req = rq.get(url)
    html = req.text
    soup = BeautifulSoup(html, "lxml")
    tables=soup.find_all("table", width = "100%")
    
    # 
    num = 1
    sub =pd.DataFrame(columns = ["Course", "Term","Day", "Begin", "End", "Building", "Room","Prerequisites", "Cross"])

    for a in tables:


        refresh = [[]]
        refresh2 = [[]]
        string = str(num)
        if num < 10:
            string = "0"+string
        # address
        Course=("rpResults_ctl"+string+"_lblCNum")
        Term = ("rpResults_ctl"+string+"_lblTerm")
        status = ("rpResults_ctl"+string+"_lblStatus")
        pre = ("rpResults_ctl"+string+"_lblPrerequisites")
        cross = ("rpResults_ctl"+string+"_lblCrossListed")
        # 스케줄 
        
        # 수업 데이터 없는거 스킵
        nineteen = a.find("span", id = Term).text
        if a.find("td", width = "20%") == None:
            num += 2
            continue
        # 2019 만 가져오기 
        if "2019" not in nineteen:
            num += 2
            continue
        Status = a.find("span", id = status)
        # cancelled or closed skip
        if "Cancelled" in Status or "Closed" in Status:
            num +=2 
            continue
        # 데이터 정리 
        # Course and Term
        refresh[0].append(a.find("span", id = Course).text)
        refresh[0].append(a.find("span", id = Term).text)
        schedule =a.find_all("td", width = "20%")
        
        
        # pre를 위한 정규표현식
        pattern2 = re.compile("[A-Z]{3}\s?\d{3}[A-Z]?")
        
        #  1 줄일때
        if len(schedule) <= 5:
            for s in schedule:
                # 시간표 & 장소
                refresh[0].append(s.text.strip())
            # Prerequisites
            if a.find("span", id = pre) == None or "None" in a.find("span", id = pre) or "none" in a.find("span", id = pre):
                refresh[0].append("")
            else:
                prere=a.find("span", id = pre).text
                refresh[0].append(pattern2.findall(prere))
            
            # Cross
            if a.find("span", id = cross) == None:
                refresh[0].append("")
            else:
                refresh[0].append(a.find("span", id = cross).text)
                
            # 정리    
            result0 = pd.DataFrame(data = refresh,columns = ["Course", "Term","Day", "Begin", "End", "Building", "Room","Prerequisites", "Cross"])
            sub = sub.append(result0, ignore_index= True)
            
        #  2 줄일때  
        else:
            # Course and Term
            refresh2[0].append(a.find("span", id = Course).text)
            refresh2[0].append(a.find("span", id = Term).text)
            d = 0
            #  시간표 & 장소
            while d < 5:
                s = schedule[d]
                refresh[0].append(s.text.strip())
                d += 1
            while d < 10 and d>=5:
                s = schedule[d]
                refresh2[0].append(s.text.strip())
                d+=1
            # Prerequisites 1 줄 
            if a.find("span", id = pre) == None or "None" in a.find("span", id = pre) or "none" in a.find("span", id = pre):
                refresh[0].append("")
            else:
                prere=a.find("span", id = pre).text
                refresh[0].append(pattern2.findall(prere))
            # for  pre 2번째 줄 
            if a.find("span", id = pre) == None or "None" in a.find("span", id = pre) or "none" in a.find("span", id = pre):
                refresh2[0].append("")
            else:
                prere=a.find("span", id = pre).text
                refresh2[0].append(pattern2.findall(prere))
            
            #  Cross 1줄 
            if a.find("span", id = cross) == None:
                refresh[0].append("")
            else:
                refresh[0].append(a.find("span", id = cross).text)
                
            # Cross 2줄 
            if a.find("span", id = cross) == None:
                refresh2[0].append("")
            else:
                refresh2[0].append(a.find("span", id = cross).text)
                
            
            result0 = pd.DataFrame(data = refresh,columns = ["Course", "Term","Day", "Begin", "End", "Building", "Room","Prerequisites", "Cross"])
            result1 = pd.DataFrame(data = refresh2,columns = ["Course", "Term","Day", "Begin", "End", "Building", "Room","Prerequisites", "Cross"])
            sub = sub.append(result0, ignore_index = True)
            sub = sub.append(result1, ignore_index = True)
        
        num += 2 
            
       
    return sub

In [ ]:
All_subject= pd.DataFrame(columns = ["Course", "Term","Day", "Begin", "End", "Building", "Room","Prerequisites", "Cross"])
for sub in clean_sub:
    All_subject.append(Crawling_cdcs(sub), ignore_index = True)

In [24]:
All_subject.to_csv("All_subject")